In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import scipy.integrate
import astropy.units as u
import GCRCatalogs
import pandas as pd

%matplotlib inline

try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

import sys
import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

catalog = 'skysim5000_v1.1.1'
skysim_cat = GCRCatalogs.load_catalog(catalog)
cosmo_ss  = skysim_cat.cosmology

from astropy.io import fits
from astropy.table import Table


ncdata_fits = fits.open('skysim5000_halos_m200c_13.fits')
#ncdata_fits.info()

ncdata_data =  ncdata_fits[1].data

ncdata_Table = Table(ncdata_data)
ncdata_Table
ncdata_Table['M200c'] = ncdata_Table['baseDC2/sod_halo_mass']/cosmo_ss.h # sod_halo_mass = M200,c in Msun/h, needs conversion

FileNotFoundError: [Errno 2] No such file or directory: 'skysim5000_halos_m200c_13.fits'

In [ ]:
def mcmc(zmin,zmax,mmin,mult):
    mmax = 1.e15
    from astropy.io import fits
    from astropy.table import Table


    ncdata_fits = fits.open('skysim5000_halos_m200c_13.fits')
    #ncdata_fits.info()

    ncdata_data =  ncdata_fits[1].data

    ncdata_Table = Table(ncdata_data)
    ncdata_Table
    ncdata_Table['M200c'] = ncdata_Table['baseDC2/sod_halo_mass']/cosmo_ss.h # sod_halo_mass = M200,c in Msun/h, needs conversion

    filt3 = ncdata_Table['M200c'] >= mmin
    filt4 = ncdata_Table['M200c'] < mmax
    filt5 = ncdata_Table['redshift_true'] >= zmin
    filt6 = ncdata_Table['redshift_true'] < zmax

    filt = filt3 * filt4 * filt5 * filt6

    data_m_ss = ncdata_Table['M200c'][filt] # M200,c [Msun]
    data_z_ss = ncdata_Table['redshift_true'][filt]
    N_cl = len(data_m_ss)
    print(f'There are {N_cl} halos in this mass (Msod) and redshift range')
    
    #cosmological model
    cosmo = Nc.HICosmoDEXcdm()
    reion = Nc.HIReionCamb.new () 
    prim = Nc.HIPrimPowerLaw.new () 

    cosmo.add_submodel (reion)
    cosmo.add_submodel (prim)

    dist = Nc.Distance.new (2.0)

    tf = Nc.TransferFunc.new_from_name ("NcTransferFuncEH")

    psml = Nc.PowspecMLTransfer.new (tf)

    #psml = Nc.PowspecMLCBE.new ()
    psml.require_kmin (1.0e-6)
    psml.require_kmax (1.0e3)

    psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
    psf.set_best_lnr0 ()
    cosmo.props.H0      = cosmo_ss.H0.value
    cosmo.props.Omegab  = cosmo_ss.Ob0
    cosmo.props.Omegac  = cosmo_ss.Odm0
    cosmo.props.Omegax  = cosmo_ss.Ode0

    cosmo.omega_x2omega_k ()
    cosmo.param_set_by_name ("Omegak", 0.0)

    prim.props.n_SA = cosmo_ss.n_s
    print(cosmo_ss.sigma8,cosmo.sigma8(psf),cosmo.Omega_k0())

    old_amplitude = math.exp (prim.props.ln10e10ASA)
    prim.props.ln10e10ASA = math.log ((cosmo_ss.sigma8 / cosmo.sigma8(psf))**2 * old_amplitude)
    print(cosmo_ss.sigma8, cosmo.sigma8(psf))
    
    #mass and multiplicty functions
    if mult == 'Tinker':
        mulf = Nc.MultiplicityFuncTinker.new_full (Nc.MultiplicityFuncMassDef.CRITICAL,200)
    if mult == 'Bocquet':
        mulf = Nc.MultiplicityFuncBocquet.new_full (Nc.MultiplicityFuncMassDef.CRITICAL,0,200)
    if mult != 'Tinker' and mult != 'Bocquet' : 
        raise TypeError('only Tinker or Bocquet')
    mf = Nc.HaloMassFunction.new (dist, psf, mulf)
    
    #proxies
    lnM_min = math.log (mmin)
    lnM_max = math.log (mmax)
    cluster_m = Nc.ClusterMass.new_from_name ("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (lnM_min, lnM_max))
    cluster_z = Nc.ClusterRedshiftNodist(z_min = zmin, z_max=zmax)
    
    #dataset
    cad = Nc.ClusterAbundance.new (mf, None)
    ncdata = Nc.DataClusterNCount.new (cad, "NcClusterRedshiftNodist", "NcClusterMassNodist")
    mset = Ncm.MSet.new_array ([cosmo, cluster_z, cluster_m])
    
    ncdata.set_lnM_true(Ncm.Vector.new_array(np.log (data_m_ss)))
    ncdata.set_z_true(Ncm.Vector.new_array(data_z_ss))
    ncdata.set_lnM_obs (Ncm.Matrix.new_array (data_m_ss,1))
    ncdata.set_z_obs (Ncm.Matrix.new_array (data_z_ss,1))
    mf.set_area_sd (skysim_cat.sky_area)
    ncdata.true_data (True)
    ncdata.set_init(True)
    cosmo.props.Omegac_fit    = True
    cosmo.props.Omegab_fit    = False
    prim.props.ln10e10ASA_fit = True

    dset = Ncm.Dataset.new ()
    dset.append_data (ncdata)

    lh = Ncm.Likelihood.new (dset)
    #fit
    
    fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_FORWARD)
    m2lnL_bf = fit.m2lnL_val()
    fit.log_info ()
    fit.obs_fisher ()
    fit.log_covar ()
    
    cosmo.props.Omegac  = cosmo_ss.Odm0
    prim.props.ln10e10ASA = math.log ((cosmo_ss.sigma8 / cosmo.sigma8(psf))**2 * old_amplitude)
    m2lnL_f = fit.m2lnL_val()
    likeratio = m2lnL_f/m2lnL_bf
    
    
    #mcmc
    Ncm.func_eval_set_max_threads (150)
    Ncm.func_eval_log_pool_stats ()
    mfunc_oa = Ncm.ObjArray.new ()

    mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
    mfunc_Omegam = Ncm.MSetFuncList.new ("NcHICosmo:Omega_m0", None)

    mfunc_oa.add (mfunc_sigma8)
    mfunc_oa.add (mfunc_Omegam)

    print (mfunc_sigma8.eval0 (mset))
    print (mfunc_Omegam.eval0 (mset))
    init_sampler = Ncm.MSetTransKernGauss.new (0)
    init_sampler.set_mset (mset)
    init_sampler.set_prior_from_mset ()
    init_sampler.set_cov_from_rescale (1.0) #1
    sampler = 'apes'
    nwalkers = int (math.ceil (500)) #500
    ssize    = 15000 #1000000

    if sampler == 'apes':
      walker = Ncm.FitESMCMCWalkerAPES.new (nwalkers, mset.fparams_len ())
    elif sampler == "stretch":
      walker = Ncm.FitESMCMCWalkerStretch.new (nwalkers, mset.fparams_len ())
    esmcmc  = Ncm.FitESMCMC.new_funcs_array (fit, nwalkers, init_sampler, walker, Ncm.FitRunMsgs.SIMPLE, mfunc_oa)
    esmcmc.set_data_file ("z_%.1f_%.1f_M_%.e_%.e_%s.fits" % (zmin, zmax,mmin,mmax,mult))
    esmcmc.set_nthreads (150)
    esmcmc.start_run ()
    esmcmc.run (ssize / nwalkers)
    esmcmc.end_run ()
    esmcmc.mean_covar ()
    fit.log_covar ()
    
    print('# case z_%.1f_%.1f_M_%.e_%.e_%s  -2lnL_Bestfit = %f , -2lnL_Fiducial = %f , then  -2lnL_Fiducial/-2lnL_Bestfit = %f' % (zmin, zmax, mmin, mmax, mult, m2lnL_bf, m2lnL_f,likeratio))
    
def corner_plot(esmcmc_fits):
    
    nwalkers = 50
    burnin   = 80
    mcat = Ncm.MSetCatalog.new_from_file_ro (esmcmc_fits, nwalkers * burnin)
    from chainconsumer import ChainConsumer

    c = ChainConsumer()

    nwalkers = mcat.nchains ()
    m2lnL = mcat.get_m2lnp_var () 

    rows = np.array ([mcat.peek_row (i).dup_array () for i in range (nwalkers * 10, mcat.len ())])
    params = ["$" + mcat.col_symb (i) + "$" for i in range (mcat.ncols ())]
    posterior = -0.5 * rows[:,m2lnL]

    rows   = np.delete (rows,   m2lnL, 1)
    params = np.delete (params, m2lnL, 0)

    indices = [1,0]

    rows   = rows[:,indices]
    params = params[indices]

    c.add_chain(rows, posterior = posterior, parameters=list(params))
    c.configure (kde = True, label_font_size=11, sigma2d=False, sigmas = [1, 2,3], spacing = 0.0, tick_font_size=11, usetex=False)


    plot_args = {}
    plot_args['truth'] = [cosmo_ss.Odm0 + cosmo_ss.Ob0, cosmo_ss.sigma8]
    plot_args['extents'] = [(0.2, 0.32),(0.75, 0.9)]
    
    plot_name = esmcmc_fits.replace('fits','png')
    fig = c.plotter.plot(**plot_args)
    c.plotter.plot(filename=plot_name, figsize=0.75, **plot_args)
    fig.set_size_inches(14.0, 14.0)
    